<a href="https://colab.research.google.com/github/onuralpArsln/finTech/blob/main/getDataFromBinanceWithoutAccount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Gerekli Kütüphane yoksa yükle

In [ ]:
!pip install python-binance -q

Datayı hesap bilgilisi olmadan almak burası, ancak colabda çalışmama ihtimali yüksek bi sebepten dolayı colab istekleri yetkisiz gözüküyor

In [4]:
import requests
import time


def getSevenDayData(lastDay=1):
    url = 'https://api.binance.com/api/v3/klines'
    current_time = int(time.time() * 1000)
    startDay = current_time - ((lastDay+7) * 24 * 60 * 60 * 1000)
    endDay = current_time - (lastDay * 24 * 60 * 60 * 1000)

    params = {
    'symbol': 'BTCUSDT',      # The trading pair you're interested in
    'interval': '1h',         # 1-hour candlestick data (hourly prices)
    'startTime': startDay,  # Data from 30 days ago
    'endTime': endDay,    # Data until 20 days ago
    }

    response = requests.get(url, params=params)
    data = response.json()

    result=[]

    for candle in data:
        close_price = candle[4]
        result.append(close_price)
    
    return result




test et

In [5]:
print(getSevenDayData())

['60138.14000000', '59973.99000000', '60070.20000000', '59851.09000000', '59706.39000000', '59794.03000000', '59769.11000000', '59914.00000000', '59876.00000000', '59981.99000000', '59885.26000000', '59786.00000000', '59810.00000000', '60024.01000000', '60036.00000000', '60006.97000000', '59993.03000000', '60025.95000000', '60235.22000000', '60238.01000000', '60155.93000000', '60117.99000000', '60160.00000000', '60164.01000000', '60196.00000000', '60069.99000000', '60016.00000000', '59991.30000000', '60000.51000000', '60104.39000000', '60189.98000000', '60279.52000000', '60335.41000000', '59950.05000000', '59814.07000000', '59971.09000000', '59878.22000000', '59795.99000000', '59651.69000000', '59430.31000000', '59132.00000000', '58666.36000000', '58607.07000000', '58263.99000000', '58575.09000000', '58352.00000000', '58670.00000000', '58707.43000000', '58911.10000000', '58924.98000000', '58636.54000000', '58749.99000000', '58663.99000000', '58527.99000000', '58497.10000000', '57604.00

toplanan bilgiyi csv formatına çevirme kodu 

In [16]:
import csv

data = getSevenDayData(28)


with open("fourWeek.csv", "w", newline="") as file:
    writer = csv.writer(file)
    for item in data:
        writer.writerow([item])

csv bilgilerini numpy arraya çevirme kodu

In [ ]:
!pip install numpy -q

In [13]:
import numpy as np

# Load CSV into numpy array
array_from_csv = np.loadtxt("output.csv", delimiter=",")

print(array_from_csv[0])
print(array_from_csv[2])
print(array_from_csv[3])


60138.14
60070.2
59851.09


Colab kullanırken csvleri otomatik çekme kodu